# Nhóm hướng đến bài toán dự đoán giá đất nền ở thành phố Đà nẵng

### Import library

In [1]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from requests_html import HTML
from requests_html import HTMLSession
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import sys

## Crawl link from nhatot.vn

In [2]:
arr = []
try:
    for page in range(85): #140
        page= str(page+1)
        link ="https://www.nhatot.com/mua-ban-dat-da-nang?page="+page
        session = HTMLSession()
        driver = webdriver.Chrome()
        driver.get(link)
        time.sleep(1)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # session = HTMLSession()
        # r = session.get(link)
        # soup = BeautifulSoup(r.text,'html.parser')
        # a = soup.find_all("a",class_='AdItem_adItem__gDDQT')
        a = soup.find_all('a', {'class': 'AdItem_adItem__gDDQT'})

        for i in range(len(a)):
            link ="https://nhatot.com/"+a[i]['href']
            arr.append(link)
    df = pd.DataFrame(arr,columns=['links'])
except KeyboardInterrupt:
    pass  # Xử lý ngoại lệ khi người dùng gián đoạn chương trình
finally:
    df = pd.DataFrame(arr, columns=['links'])

### Export file link_nhatot.csv

In [3]:
df.to_csv('.\\raw data\\data_test\\link_nhatot_test.csv', index=True)
print("hoàn thành")

hoàn thành


### View a head of file nhatot.csv

In [4]:
frame = pd.read_csv('.\\raw data\\data_test\\link_nhatot_test.csv')
frame.head()

,Unnamed: 0,links
0,0,https://nhatot.com//mua-ban-dat-huyen-hoa-vang...
1,1,https://nhatot.com//mua-ban-dat-quan-ngu-hanh-...
2,2,https://nhatot.com//mua-ban-dat-quan-lien-chie...
3,3,https://nhatot.com//mua-ban-dat-quan-ngu-hanh-...
4,4,https://nhatot.com//mua-ban-dat-quan-lien-chie...


### Crawl data from each link

In [5]:
Address,Price = [],[]
Area, Price_m2, Direction =[], [], []
Land_type =[] 
Width, Length = [], []


In [6]:
crawl_flag = True
def getdata(link):
    global crawl_flag
    session = HTMLSession()
    driver = webdriver.Chrome()
    driver.get(link)
    # Chờ trang web tải hoàn thành trong 10 giây
    time.sleep(1)
    
    # Kiểm tra xem trang web đã tải hoàn toàn hay chưa
    if driver.page_source:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        #Time
        span_times = []
        span_times = soup.find_all('div', {'class': 'AdDecriptionVeh_date__SpYR1'})
        if span_times:
            span_time = span_times[1].get_text(strip=True)
            if '2 tuần trước' in span_time:
                crawl_flag = False

        if crawl_flag:
             # Price
            price_tag = soup.find('span', itemprop='price')
            if price_tag:
                price = price_tag.text.strip()
            else:
                price = "NaN"
            Price.append(price)

            #Address
            span_address = soup.find('span', {'class': 'fz13'})
            if span_address:
                address = span_address.text.strip()
            else:
                address='NaN'
            Address.append(address)
            # Đóng trình duyệt sau khi hoàn thành


            #Area
            area_tag = soup.find('span', itemprop='size')
            if area_tag:
                area = area_tag.text.strip()
            else:
                area = "NaN"
            Area.append(area)

            #Price_m2
            price_m2_tag = soup.find('span', itemprop='price_m2')
            if price_m2_tag:
                price_m2 = price_m2_tag.text.strip()
            else:
                price_m2 = "NaN"
            Price_m2.append(price_m2)

            #Direction
            direction_tag = soup.find('span', itemprop='direction')
            if direction_tag:
                direction = direction_tag.text.strip()
            else:
                direction = "NaN"
            Direction.append(direction)


            #Land_type
            land_type_tag = soup.find('span', itemprop='land_type')
            if land_type_tag:
                land_type= land_type_tag.text.strip()
            else:
                land_type = "NaN"
            Land_type.append(land_type)

            #Width
            width_tag = soup.find('span', itemprop='width')
            if width_tag:
                width= width_tag.text.strip()
            else:
                width = "NaN"
            Width.append(width)

            #Length
            length_tag = soup.find('span', itemprop='length')
            if length_tag:
                length = length_tag.text.strip()
            else:
                length = "NaN"
            Length.append(length)
    df = {'Gia': Price, 'Dia chi': Address, 'Dien tich': Area, 'Gia/m2': Price_m2, 'Huong dat':Direction,
          'Loai hinh dat' :Land_type,
          'Chieu ngang': Width, 'Chieu dai': Length
        }
    df = pd.DataFrame(df)
    df.dropna(inplace=True)
    df.to_csv('.\\raw data\\data_test\\raw_data_test.csv', encoding="utf-8-sig",index=False)

    driver.quit()

# Thực hiện trích xuất dữ liệu từ mỗi liên kết trong dataframe
for link in frame['links']:
    if not crawl_flag:
        break  # Nếu crawl_flag=False, thoát khỏi vòng lặp
    data = getdata(link)

- **Gia**: giá bán của đất nền

- **Dia chi**: địa chỉ của đất nền ở thành phố Đà Nẵng

- **Dien tich**: diện tích của đất nền

- **Gia/m2**: giá bán trung bình mỗi mét vuông của đất nền

- **Huong dat**: hướng của đất nền

- **Loai hinh dat**: Phân loại đất nền dựa trên việc sử dụng đất và các điều kiện liên quan đến pháp lý, quy hoạch và mục đích sử dụng

- **Chieu ngang**: chiều ngang(mặt tiền) của đất nền

- **Chieu dai**: chiều dài của đất nền